# Progetto DSIM - Image acquisition

---



Autori: Ginevra Mariani | Lorenzo Mora | Confalonieri Riccardo <br>
E-mail: g.mariani34@campus.unimib.it | l.mora4@campus.unimib.it | r.confalonieri5@campus.unimib.it

Questo notebook implementa una funzione per scattare in automatico le foto necessarie per il progetto. Si è deciso di variare sia l'espressione che le condizioni di luce durante l'acquisizione. <br>
Per questo notebook è necessario l'utilizzo della webcam ed è quindi importante eseguirlo localmente e non tramite Colab!

## Import packages

In [ ]:
import warnings
import cv2 as cv
# per gestire lo scatto della foto dal video e la scrittura del file
from cv2 import VideoCapture, imwrite
# per gestire sleep automatico
import time
# per gestire ii path delle cartelle
import os
# per pulire output console
from IPython.display import clear_output

## Acquisizione immagini da streaming video
Per ogni utente vengono salvate $100$ immagini diverse, le immagini vengono scattate ogni due secondi. Durante l'attesa viene visualizzata una finestra esterna con lo streaming dei frame per facilitare l'acquisizione. Vengono inoltre visualizzati dei messaggi per capire quanto tempo resta prima del prossimo scatto

In [ ]:
def check_folder(folder_name):
  """
  Verifica se esiste una cartella con il nome specificato nel path locale,
  se non esiste la crea.
  
  :param str folder_name: Nome della cartella
  """

  # se non esiste la creo
  if not os.path.exists(folder_name):
      print("----- Creating folder: ./" + str(folder_name))
      os.mkdir(folder_name)

# funzione per il salvataggio del file
def save_frame(img, name):
  """
  Effettua il salvataggio di un immagine su file
 
  :param numpy.ndarray img: Immagine da salvare
  :param str name: nome dell'immagine

  :return istante in cui viene salvata l'immagine
  :rtype: float
  """

  # definisco il filename come foldername_timestamp
  t = time.localtime()
  timestamp = time.strftime('%m-%d-%Y_%H%M%S', t)
  img_name = name + "_" + str(timestamp) + ".jpg"

  # scrittura dell'immagine su file
  imwrite(name + "/" + img_name, img)
  print("Immagine", img_name, 'salvata!')
  
  return time.time() # ritorno il momento in cui viene salvata l'img

In [ ]:
def img_acquisition_windows(name, num_img = 100, wait_time = 2):
    """
    Visualizza una finestra con la riproduzione streaming catturata dalla camera principale
    del pc. Il frame catturato viene quindi salvato nel path locale col nome definito
    
    :param str name: nome cartella dove salvare gli audio il path sarà (./nome_cartella)
    :param int num_img: numero di immagini da catturare
    :param int wait_time: secondi di attesa tra un immagine e l'altra
    """

    # verifico se esiste la cartella di destinazione delle foto
    check_folder(name)
    
    # inizializzo la camera
    cam = VideoCapture(0)
        
    print('-------------- Inizio acquisizione --------------')
    
    i = 0
    
    # salvo il tempo di inizio, scattero un immagine ogni tot secondi 
    old_time = time.time()
    salva = False # boolean per sapere se salvare il frame o no
    
    while(True):
        # Catturo un nuovo frame
        r, frame = cam.read()
        
        # se sono passati tot secondi salvo il frame
        if time.time() - old_time >= wait_time:
            salva = True
        # se la prossima immagine verrà salvata visualizzo il messaggio a video           
        elif time.time() - old_time >= wait_time - 0.3: 
            cv.putText(frame, 
                       ("Scatto immagine numero: " + str(i)), 
                       ((5, 30)), 
                       cv.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),2)     
        # visualizzo quanti secondi rimangono prima del prossimo scatto  
        else:
            time_next_img = round(wait_time - (time.time() - old_time),3)
            cv.putText(frame, 
                       ("Tempo prima della prossima immagine: " + str(time_next_img)), 
                       ((5, 30)), 
                       cv.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2)
  
        # se sono passatti tot secondi salvo la nuova immagine
        if salva:
            # Salvo il contenuto del frame
            old_time = save_frame(frame, name)
            i = i + 1 # aumento il contatore delle immagini
            # stampa di controllo
            print('-------- Acquisita immagine numero:', str(i), 
                  ' -------- Mancano', str(num_img-i), 'immagini da acquisire')
            
            salva = False

        
        # Visualizzo il frame nella finestra esterna
        cv.imshow('Acquisizione immagini', frame)
            
        # Interrupt when Q key is pressed
        if cv.waitKey(20) & 0xFF == ord('q'):
            break
        # interrompo se ho catturato il numero di immagini richieste
        if i == num_img:
            break
        
        # ogni 5 immagini salvate pulisco l'output per semplicità
        if i%5 == 0:
            clear_output()
    
    # rilascio la cam e distruggo le finestre aperte
    cam.release()
    cv.destroyAllWindows()

In [ ]:
img_acquisition_windows('Riccardo') # acquisisco le foto

In [2]:
# download notebook in .html extension
%%shell
jupyter nbconvert --to html '/content/drive/MyDrive/Confalonieri_Mariani_Mora_DSIM/Notebook/Processing-2D/1_ImageAcquisition.ipynb'

[NbConvertApp] Converting notebook /content/drive/MyDrive/Confalonieri_Mariani_Mora_DSIM/Notebook/Processing-2D/1_ImageAcquisition.ipynb to html
[NbConvertApp] Writing 291874 bytes to /content/drive/MyDrive/Confalonieri_Mariani_Mora_DSIM/Notebook/Processing-2D/1_ImageAcquisition.html
